In [ ]:
# default_exp sumstat

# Sumstat module

> read and extract Sumstat

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import yaml
import numpy as np
import pandas as pd
from scipy.stats import norm
from cugg.utils import *

In [ ]:
#export
def p2z(pval,beta,twoside=True):
    if twoside:
        pval = pval/2
    z=np.abs(norm.ppf(pval))
    ind=beta<0
    z[ind]=-z[ind]
    return z
    
class Sumstat:
    def __init__(self,sumstat_path,config_file=None,rename=True):
        self.ss = self.read_sumstat(sumstat_path,config_file,rename)
        
    def __repr__(self): 
        return "sumstat:% s" % (self.ss)
        
        #functions to read sumstats
    def read_sumstat(self,file, config_file,rename):
        if config_file is not None:
            config_file = yaml.safe_load(open(config_file, 'r'))
        return read_sumstat(file,config_file,rename)
    
    def extractbyregion(self,region):
        sumstats = self.ss
        idx = (sumstats.CHR == region[0]) & (sumstats.POS >= region[1]) & (sumstats.POS <= region[2])
        print('this region',region,'has',sum(idx),'SNPs in Sumstat')
        self.ss = sumstats[idx]
        
    def extractbyvariants(self,variants,notin=False):
        idx = self.ss.SNP.isin(variants)
        if notin:
            idx = idx == False
        #update sumstats
        self.ss = self.ss[idx]
        
    def calculateZ(self):
        self.ss['Z'] = list(p2z(self.ss.P,self.ss.BETA))
        
    def match_ss(self,bim):
        self.ss = check_ss1(self.ss,bim)
        
        

In [ ]:
#export
def read_sumstat(file, config,rename=True):
    try:
        sumstats = pd.read_csv(file, compression='gzip', header=0, sep='\t', quotechar='"')
    except:
        sumstats = pd.read_csv(file, header=0, sep='\t', quotechar='"')
    if config is not None:
        try:
            ID = config.pop('ID').split(',')
            sumstats = sumstats.loc[:,list(config.values())]
            sumstats.columns = list(config.keys())
            sumstats.index = namebyordA0_A1(sumstats[ID],cols=ID)
        except:
            raise ValueError(f'According to config_file, input summary statistics should have the following columns: %s' % list(config.values()))
        sumstats.columns = list(config.keys())
    if rename:
        sumstats.SNP = 'chr'+sumstats.CHR.astype(str).str.strip("chr") + ':' + sumstats.POS.astype(str) + '_' + sumstats.A0.astype(str) + '_' + sumstats.A1.astype(str)
    sumstats.CHR = sumstats.CHR.astype(str).str.strip("chr").astype(int)
    sumstats.POS = sumstats.POS.astype(int)
    if "GENE" in sumstats.columns.values():
        sumstats.index = namebyordA0_A1(sumstats[["GENE","CHR","POS","A0","A1"]],cols=["GENE","CHR","POS","A0","A1"])
    else:
        sumstats.index = namebyordA0_A1(sumstats[["CHR","POS","A0","A1"]],cols=["CHR","POS","A0","A1"])
    return sumstats

## Sumstat 2 vcf
Following function take a sumstat pandas.df object that were processed and with correct headers ('CHR', 'POS', 'SNP', 'A0', 'A1',"STAT","SE","P"). and then change it into a vcf file with 1 samples. When the sumstat have gene name in it, the gene name will be saved to the info field.

In [ ]:
#export
def ss_2_vcf(ss_df,name = "name"):
    ## Geno field
    df = pd.DataFrame()
    if "SNP" not in ss_df.columns:
        ss_df['SNP'] = 'chr'+ss_df.CHR.astype(str).str.strip("chr") + ':' + ss_df.POS.astype(str) + '_' + ss_df.A0.astype(str) + '_' + ss_df.A1.astype(str)
    df[['#CHROM', 'POS', 'ID', 'REF', 'ALT']] = ss_df[['CHR', 'POS', 'SNP', 'A0', 'A1']].sort_values(['CHR', 'POS'])
    ## Info field(Empty)
    df['QUAL'] = "."
    df['FILTER'] = "PASS"
    df['INFO'] = "."
    fix_header = ["SNP","A1","A0","POS","CHR","STAT","SE","P"]
    header_list = []
    if "GENE" in ss_df.columns:
        df['ID'] = ss_df['GENE'] + ":" + ss_df['SNP']
        df['INFO'] = "GENE=" + ss_df["GENE"]
        fix_header = ["GENE","SNP","A1","A0","POS","CHR","STAT","SE","P"]
        header_list = ['##INFO=<ID=GENE,Number=1,Type=String,Description="The name of genes">']
    ### Fix headers
    import time
    header = '##fileformat=VCFv4.2\n' + \
    '##FILTER=<ID=PASS,Description="All filters passed">\n' + \
    f'##fileDate={time.strftime("%Y%m%d",time.localtime())}\n'+ \
    '##FORMAT=<ID=STAT,Number=1,Type=Float,Description="Effect size estimate relative to the alternative allele">\n' + \
    '##FORMAT=<ID=SE,Number=1,Type=Float,Description="Standard error of effect size estimate">\n' + \
    '##FORMAT=<ID=P,Number=1,Type=Float,Description="The Pvalue corresponding to ES">\n' 
    ### Customized Field headers
    for x in ss_df.columns:
        if x not in fix_header:
            Prefix = f'##FORMAT=<ID={x},Number=1,Type='
            Type = str(type(ss_df[x][0])).replace("<class \'","").replace("'>","").replace("numpy.","").replace("64","").capitalize().replace("Int","Integer")
            Surfix = f',Description="Customized Field {x}">'
            header_list.append(Prefix+Type+Surfix)
    ## format and sample field
    df['FORMAT'] = ":".join(["STAT","SE","P"] + ss_df.drop(fix_header,axis = 1).columns.values.tolist())
    df[f'{name}'] = ss_df['STAT'].astype(str) + ":" + ss_df['SE'].astype(str) + ":" + ss_df['P'].astype(str) + ":" + ss_df.drop(fix_header,axis = 1).astype(str).apply(":".join,axis = 1)
    ## Rearrangment
    df = df[['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO','FORMAT',f'{name}']]
    # Add headers
    header = header + "\n".join(header_list) + "\n"
    return df,header

## Example input and output for ss_2_vcf()

In [164]:
test

,GENE,SNP,TSS_D,AF,MA_S,MA_F,P,STAT,SE,A1,A0,POS,CHR
0,DOC2B,chr17:150509_TA_T,-31141,0.185990,135,154,2.767300e-09,-0.353553,0.057822,T,TA,150509,chr17
1,DOC2B,chr17:151035_C_T,-30615,0.185990,136,154,2.005370e-09,-0.360252,0.058372,T,C,151035,chr17
2,DOC2B,chr17:151041_A_G,-30609,0.140097,102,116,8.754326e-03,-0.175506,0.066545,G,A,151041,chr17
3,DOC2B,chr17:151596_G_A,-30054,0.096618,77,80,2.739677e-01,-0.088473,0.080737,A,G,151596,chr17
4,DOC2B,chr17:151872_T_C,-29778,0.115942,92,96,9.243513e-02,0.132022,0.078229,C,T,151872,chr17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,DOC2B,chr17:281918_A_G,100268,0.335749,234,278,3.852610e-02,0.107802,0.051887,G,A,281918,chr17
996,DOC2B,chr17:281965_T_G,100315,0.289855,206,240,8.156741e-03,0.140386,0.052740,G,T,281965,chr17
997,DOC2B,chr17:282037_G_A,100387,0.010870,9,9,6.988573e-02,0.443654,0.243950,A,G,282037,chr17
998,DOC2B,chr17:282127_G_A,100477,0.160628,122,133,8.626497e-01,0.011448,0.066118,A,G,282127,chr17


In [163]:
print(ss_2_vcf(test)[1])
ss_2_vcf(test)[0]

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=20220322
##FORMAT=<ID=ES,Number=A,Type=Float,Description="Effect size estimate relative to the alternative allele">
##FORMAT=<ID=SE,Number=A,Type=Float,Description="Standard error of effect size estimate">
##FORMAT=<ID=P,Number=A,Type=Float,Description="The Pvalue corresponding to ES">
##FORMAT=<ID=TSS_D,Number=A,Type=Int,Description="Customized Field TSS_D
##FORMAT=<ID=AF,Number=A,Type=Float,Description="Customized Field AF
##FORMAT=<ID=MA_S,Number=A,Type=Int,Description="Customized Field MA_S
##FORMAT=<ID=MA_F,Number=A,Type=Int,Description="Customized Field MA_F


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,name
0,chr17,150509,chr17:150509_TA_T,TA,T,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,-0.35355327:0.05782158:2.7672995977620612e-09-...
1,chr17,151035,chr17:151035_C_T,C,T,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,-0.3602523:0.05837245:2.005370139560338e-09-30...
2,chr17,151041,chr17:151041_A_G,A,G,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,-0.17550638:0.06654477:0.0087543256724295-3060...
3,chr17,151596,chr17:151596_G_A,G,A,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,-0.08847341:0.08073738:0.2739676780558129-3005...
4,chr17,151872,chr17:151872_T_C,T,C,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,0.13202243:0.07822862:0.0924351300538036-29778...
...,...,...,...,...,...,...,...,...,...,...
995,chr17,281918,chr17:281918_A_G,A,G,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,0.107802205:0.051887173:0.03852610270631691002...
996,chr17,281965,chr17:281965_T_G,T,G,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,0.14038579:0.052740265:0.008156741448439610031...
997,chr17,282037,chr17:282037_G_A,G,A,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,0.4436538:0.24394979:0.0698857298190166100387:...
998,chr17,282127,chr17:282127_G_A,G,A,.,PASS,GENE = DOC2B,STAT:SE:P:TSS_D:AF:MA_S:MA_F,0.011447639:0.066118255:0.86264972708922861004...
